In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import hyp3_sdk as sdk
import asf_search as asf
import pyproj
import json
import rasterio as rio
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from PIL import Image
from shapely.geometry import Polygon
from config import hyp3_username, hyp3_password
from med_granules import med_granules

In [3]:
sys.path.append('../../SARSearch/src/')
from sarutils import SARUtils

In [ ]:
def merge_landmasks(mask_files, output_file):
    # Open the mask files
    src_files_to_mosaic = [rasterio.open(mask) for mask in mask_files]
    
    # Merge the masks
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    # Update the metadata
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": src_files_to_mosaic[0].crs,
        "dtype": rasterio.uint8,
        "compress": "lzw"  # Apply compression
    })

    # Write the merged mask to a new file
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

    # Close the opened files
    for src in src_files_to_mosaic:
        src.close()

if __name__ == '__main__':
    mask_files = [
        '/home/kai/datasets/W140N60_PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-proba_EPSG-4326.tif',
        '/home/kai/datasets/W120N40_PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-proba_EPSG-4326.tif',
        '/home/kai/datasets/W140N40_PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-proba_EPSG-4326.tif'
    ]
    output_file = '/home/kai/datasets/us_westcoast_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-proba_EPSG-4326.tif'
    merge_landmasks(mask_files, output_file)


In [ ]:
A733_VV = '/home/kai/datasets/jpl_slice/cali_sentinel1_frames_hyp3_processed/S1A_IW_20210204T140058_DVP_RTC20_G_gduned_A733/S1A_IW_20210204T140058_DVP_RTC20_G_gduned_A733_VV.tif'
_4BEC_VV = '/home/kai/datasets/jpl_slice/ben_kai_test/vv/S1A_IW_GRDH_1SDV_20210925T140807_20210925T140832_039839_04B67B_AC65_VV.tif'
with rio.open(_4BEC_VV) as scene:
    image_data = scene.read(1)
    no_data_value = scene.nodata
    image_data = np.where(image_data == no_data_value, np.nan, image_data)

In [ ]:
plt.figure(figsize=(50,50))
plt.imshow(image_data, cmap='gray', vmin=np.nanmin(image_data), vmax=np.nanmax(image_data))
plt.colorbar()
plt.title('GeoTIFF Visualization')
plt.show()

In [ ]:
no_data_value

In [ ]:
np.unique(image_data)

In [ ]:
sar_image_path = _4BEC_VV
landcover_tif_path = '/home/kai/datasets/us_westcoast_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-proba_EPSG-4326.tif'  # Update with the correct path

# Read the SAR image using rasterio
with rasterio.open(sar_image_path) as src:
    sar_image = src.read(1)  # Read the first band
    no_data_value = src.nodata
    sar_image = np.where(sar_image == no_data_value, np.nan, image_data)
    sar_meta = src.meta
    sar_crs = src.crs
    sar_transform = src.transform
    sar_bounds = src.bounds

# Read the landcover GeoTIFF using rasterio
with rasterio.open(landcover_tif_path) as land_src:
    landcover_data = land_src.read(1)
    land_transform = land_src.transform
    land_bounds = land_src.bounds

# Reproject landcover data if CRS does not match
if sar_crs != land_src.crs:
    from rasterio.warp import reproject, Resampling

    reprojected_landcover_data = np.empty(shape=(sar_meta['height'], sar_meta['width']), dtype=landcover_data.dtype)
    reproject(
        source=landcover_data,
        destination=reprojected_landcover_data,
        src_transform=land_transform,
        src_crs=land_src.crs,
        dst_transform=sar_transform,
        dst_crs=sar_crs,
        resampling=Resampling.nearest
    )
    landcover_data = reprojected_landcover_data

# Define land classes to mask (e.g., water bodies)
# For example, assuming water bodies are represented by value 210 in the landcover data
water_classes = [255]
land_mask = np.isin(landcover_data, water_classes, invert=True)

# Apply the land mask to the SAR image
masked_sar_image = np.where(land_mask, np.nan, sar_image)

# Save the masked SAR image back to a new file
masked_sar_image_path = 'foo.tif'
with rasterio.open(masked_sar_image_path, 'w', **sar_meta) as dst:
    dst.write(masked_sar_image, 1)

# Optional: Plot the original and masked images to verify
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Original SAR Image')
plt.imshow(sar_image, cmap='gray', vmin=np.nanmin(sar_image), vmax=np.nanmax(sar_image))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Masked SAR Image')
plt.imshow(masked_sar_image, cmap='gray', vmin=np.nanmin(masked_sar_image), vmax=np.nanmax(masked_sar_image))
plt.axis('off')

plt.show()


In [ ]:
np.unique(masked_sar_image)

In [ ]:
sns.histplot(vv_data[vv_data > -3 ])

In [ ]:
sns.histplot(vv_data[vv_data < 0][vv_data > -1])


In [ ]:
foo = vv_data[vv_data < 0]

In [ ]:
sns.histplot(foo[foo > -1])

In [6]:
sarutils = SARUtils('/home/kai/datasets/us_westcoast_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-proba_EPSG-4326.tif')

In [ ]:
sarutils.multiprocess_apply_landmask(input_dir='/home/kai/datasets/jpl_slice/ben_kai_test/vv/', 
                                     output_dir='/home/kai/datasets/jpl_slice/ben_kai_test/vv_landmask/')

In [8]:
sarutils.index_tiles(tiff_dir='/home/kai/datasets/jpl_slice/ben_kai_test/vv_landmask/',
                     output_file='/home/kai/datasets/jpl_slice/ben_kai_test/vv_landmask_tile_map')